In [3]:
library(tidyverse)

# Apply a function (or a set of functions) to a set of columns

`across()` makes it easy to apply the same transformation to multiple columns, allowing you to use `select()` semantics inside in `summarise()` and `mutate()`. 

`c_across()` is designed to work with `rowwise()` to make it easy to perform row-wise aggregations. It has two differences from `c()`:

* It uses tidy select semantics so you can easily select multiple variables. See vignette("rowwise") for more details.

* It uses vctrs::vec_c() in order to give safer outputs.

```R
across(.cols = everything(), .fns = NULL, ..., .names = NULL)

c_across(cols = everything())
```

**Arguments**
`.fns`	
Functions to apply to each of the selected columns. Possible values are:

* NULL, to returns the columns untransformed.

* A function, e.g. `mean`.

* A purrr-style lambda, e.g. `~ mean(.x, na.rm = TRUE)`

*  list of functions/lambdas, e.g. `list(mean = mean, n_miss = ~ sum(is.na(.x))`

Within these functions you can use cur_column() and cur_group() to access the current column and grouping keys respectively.

`...`	
Additional arguments for the function calls in .fns.

`.names`	
A glue specification that describes how to name the output columns. This can use {.col} to stand for the selected column name, and {.fn} to stand for the name of the function being applied. The default (NULL) is equivalent to "{.col}" for the single function case and "{.col}_{.fn}" for the case where a list is used for .fns.

`cols`, `.cols`	
<tidy-select> Columns to transform. Because across() is used within functions like summarise() and mutate(), you can't select or compute upon grouping variables.

# Examples

In [4]:
#For each Species, calculate the mean values of columns having name starts with 'Sepal'

iris %>% 
group_by(Species) %>%
summarize(across(starts_with('Sepal'), mean))

`summarise()` ungrouping output (override with `.groups` argument)


Species,Sepal.Length,Sepal.Width
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


In [7]:
#Convert factor columns to character columns
iris %>% 
mutate(across(where(is.factor), as.character)) %>% .$Species

[1] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
  [6] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [11] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [16] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [21] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [26] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [31] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [36] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [41] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [46] "setosa"     "setosa"     "setosa"     "setosa"     "setosa"    
 [51] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [56] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [61] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [66] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [71] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [76] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [81] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [86] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [91] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
 [96] "versicolor" "versicolor" "versicolor" "versicolor" "versicolor"
[101] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[106] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[111] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[116] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[121] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[126] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[131] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[136] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[141] "virginica"  "virginica"  "virginica"  "virginica"  "virginica" 
[146] "virginica"  "virginica"  "virginica"  "virginica"  "virginica"

In [9]:
# A purrr-style formula
iris %>%
  group_by(Species) %>%
  summarise(across(starts_with("Sepal"), ~mean(.x, na.rm = TRUE)))

`summarise()` ungrouping output (override with `.groups` argument)


Species,Sepal.Length,Sepal.Width
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


In [8]:
# A named list of functions
iris %>%
group_by(Species) %>%
summarize(across(starts_with('Sepal'), list(mean = mean, std = sd)))

`summarise()` ungrouping output (override with `.groups` argument)


Species,Sepal.Length_mean,Sepal.Length_std,Sepal.Width_mean,Sepal.Width_std
setosa,5.006,0.3524897,3.428,0.3790644
versicolor,5.936,0.5161711,2.770,0.3137983
virginica,6.588,0.6358796,2.974,0.3224966


In [11]:
# Use the .names argument to control the output names
iris %>%
  group_by(Species) %>%
  summarise(across(starts_with("Sepal"), mean, .names = "mean_{.col}"))

`summarise()` ungrouping output (override with `.groups` argument)


Species,mean_Sepal.Length,mean_Sepal.Width
setosa,5.006,3.428
versicolor,5.936,2.770
virginica,6.588,2.974


In [18]:
iris %>%
  group_by(Species) %>%
  summarise(across(starts_with("Sepal"), list(mean = mean, sd = sd), .names = "{.col}.{.fn}"))

`summarise()` ungrouping output (override with `.groups` argument)


Species,Sepal.Length.mean,Sepal.Length.sd,Sepal.Width.mean,Sepal.Width.sd
setosa,5.006,0.3524897,3.428,0.3790644
versicolor,5.936,0.5161711,2.770,0.3137983
virginica,6.588,0.6358796,2.974,0.3224966


Rowwise: `c_across()`

In [19]:
iris %>% head()

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [24]:
#For each row, calculate the sum of values having type numeric|
iris %>%
rowwise() %>%
summarize(total = sum(c_across(where(is.numeric))))

`summarise()` ungrouping output (override with `.groups` argument)


total
10.2
9.5
9.4
9.4
10.2
11.4
9.7
10.1
8.9
9.6
